# DynamicSpatialAttention — Satır Satır ve İnce Detaylı Açıklama

Bu not defteri **yalnızca** Spatial Attention (SA) bileşenini açıklar: `DynamicSpatialAttention`.

Amaç: Kodun her bölümünün
- ne yaptığı,
- hangi tensör şekillerini ürettiği,
- neden bu tasarımın seçildiği

konularını, Channel Attention not defterindeki seviyede ayrıntılandırmaktır.


## 1) Kod (Referans)

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F


def _make_odd(k: int) -> int:
    k = int(k)
    if k < 1:
        raise ValueError("Kernel size >= 1 olmalı.")
    return k if (k % 2 == 1) else (k + 1)


def _softplus_inverse(y: torch.Tensor, eps: float = 1e-6) -> torch.Tensor:
    return torch.log(torch.clamp(torch.exp(y) - 1.0, min=eps)) ## torch.clamp(..., min=eps) şunu garanti eder: exp(y) - 1 asla eps’ten küçük olmayacak.
# Burda eğer eps değeri eğer çok küçükse -inf ye gitmesin diye biz buraya kontrol ekliyoruz.


def _get_gate(gate: str):
    g = gate.lower()
    if g == "sigmoid":
        return torch.sigmoid
    if g == "hardsigmoid":
        return F.hardsigmoid
    raise ValueError("gate 'sigmoid' veya 'hardsigmoid' olmalı.")


class _DWPointwiseBranch(nn.Module):
    """Depthwise (in_ch -> in_ch, groups=in_ch) + Pointwise (in_ch -> 1)."""

    def __init__(self, in_ch: int, k: int, dilation: int = 1):
        super().__init__()
        k = _make_odd(k)
        dilation = int(dilation)
        if dilation < 1:
            raise ValueError("dilation >= 1 olmalı.")
        pad = dilation * (k - 1) // 2

        self.dw = nn.Conv2d(
            in_ch,
            in_ch,
            kernel_size=k,
            padding=pad,
            dilation=dilation,
            groups=in_ch,
            bias=False,
        )
        self.pw = nn.Conv2d(in_ch, 1, kernel_size=1, bias=False)

    def forward(self, s: torch.Tensor) -> torch.Tensor:
        return self.pw(self.dw(s))


class DynamicSpatialAttention(nn.Module):
    """
    Spatial Attention (SA) bloğu:
    - CoordConv: avg_map + max_map + x_coord + y_coord -> 4 kanal
    - Multi-branch (multi-scale) depthwise+pointwise conv
    - Sample-wise router ile branch ağırlıkları (B,K)
    - Temperature scaling + gate ile mask üretimi
    """

    def __init__(
        self,
        kernels=(3, 7),
        use_dilated: bool = True,
        dilated_kernel: int = 7,
        dilated_d: int = 2,
        gate: str = "sigmoid",
        temperature: float = 1.0,
        learnable_temperature: bool = False,
        eps: float = 1e-6,
        router_hidden: int = 8,
        bias: bool = True,
        return_router_weights: bool = False,
        coord_norm: str = "minus1to1",  # "minus1to1" | "0to1"

# # # x_coord: soldan sağa giden “konum” bilgisi
# # # y_coord: yukarıdan aşağı giden “konum” bilgisi
# # # coord_norm bunun ölçeğini seçiyor.

    ):
        super().__init__()

        if temperature <= 0:
            raise ValueError("temperature pozitif olmalı.")
        if router_hidden < 1:
            raise ValueError("router_hidden >= 1 olmalı.")
        if coord_norm not in ("minus1to1", "0to1"):
            raise ValueError("coord_norm 'minus1to1' veya '0to1' olmalı.") 
        
# Çoğu CNN/attention tasarımında minus1to1 daha yaygın ve güvenli.
# Çünkü 0 merkezli olması öğrenmeyi kolaylaştırabilir.
        
##         minus1to1 ne demek?        ## 
# # x_coord ve y_coord değerleri -1 ile +1 arasında olur.Model için “sağ mı sol mu” gibi yön bilgisi daha simetrik olur.Birçok modelde 0 merkezli girişler optimizasyon açısından daha rahat davranır.

# En sol: x = -1
# Orta: x ≈ 0
# En sağ: x = +1

# Benzer şekilde:
# En üst: y = -1
# Orta: y ≈ 0
# En alt: y = +1

##         0to1 ne demek?        ## 
# x_coord ve y_coord değerleri 0 ile 1 arasında olur.
# En sol: x = 0
# En sağ: x = 1
# En üst: y = 0
# En alt: y = 1

        self.eps = float(eps)
        self.return_router_weights = bool(return_router_weights)
        self.gate_fn = _get_gate(gate)
        self.coord_norm = coord_norm

        in_ch = 4  # [avg_map, max_map, x_coord, y_coord]

        # Branch pool
        ks = []
        for k in kernels:
            ks.append(_make_odd(int(k)))

        branches = []
        for k in ks:
            branches.append(_DWPointwiseBranch(in_ch=in_ch, k=k, dilation=1))

        if use_dilated:
            dk = _make_odd(int(dilated_kernel))
            dd = int(dilated_d)
            if dd < 1:
                raise ValueError("dilated_d >= 1 olmalı.")
            branches.append(_DWPointwiseBranch(in_ch=in_ch, k=dk, dilation=dd))

        self.branches = nn.ModuleList(branches)
        self.num_branches = len(self.branches)



        # Router: (B,4,H,W) -> pool -> (B,4,1,1) -> (B,hidden,1,1) -> (B,K,1,1)
        # Router’ın amacı “seçmek/karıştırmak”.

#         Buu formülün çalışması için:       #         
        # rw içinde her branch için bir ağırlık olması şart.
        # Yani rw mutlaka (B, K) olmalı.
        # Bu yüzden router’ın çıkışı (B, K) olacak şekilde tasarlanır:
        # Conv(hidden -> K)
        self.router = nn.Sequential(
            nn.AdaptiveAvgPool2d(1),
            nn.Conv2d(in_ch, router_hidden, kernel_size=1, bias=bias),
            nn.ReLU(inplace=True),
            nn.Conv2d(router_hidden, self.num_branches, kernel_size=1, bias=bias),
        )

    # Durum A: Router çıkışı 1 olsaydı
    # (B,1) çıkar
    # Bu tek sayı, tüm branch’lere aynı ağırlık gibi olur → branch karıştırma yapamazdık

    # Durum B: Router çıkışı K olursa:
    # Her branch’e bir ağırlık
    # Softmax ile normalize
    # Ağırlıklı toplam doğrudan ve temiz çalışı

        # Temperature
        self.learnable_temperature = bool(learnable_temperature)
        if self.learnable_temperature:
            t0 = torch.tensor(float(temperature))
            t_inv = _softplus_inverse(t0, eps=self.eps)
            self.t_raw = nn.Parameter(t_inv)
        else:
            self.register_buffer("T", torch.tensor(float(temperature)))

        # Coord cache (opsiyonel): (H,W,device,dtype,norm) -> (xg,yg)
        self._coord_cache = {} # Daha önce üretilmiş koordinat gridlerini saklamak

    def get_T(self) -> torch.Tensor:
        if self.learnable_temperature:
            return F.softplus(self.t_raw) + self.eps
        return self.T

    def _coords(self, B: int, H: int, W: int, device, dtype): # SA’ye “konum bilgisini” eklemek için x ve y koordinat haritaları üretir.
# # # # # # # #  “Bu piksel solda mı sağda mı?” → x_coord
# # # # # # # # “Bu piksel yukarıda mı aşağıda mı?” → y_coord

# # # # avg_map, max_map → “burada aktivasyon var mı?”
# # # # Ama bazen modelin şunu bilmesi lazım:
# # # # “Bu aktivasyon nerede?”
# # # # “Üst tarafta mı, alt tarafta mı, sol köşe mi?”

        key = (H, W, str(device), str(dtype), self.coord_norm) # “Aynı H×W, aynı device, aynı dtype ve aynı coord_norm için koordinat gridini bir kez üret, sonra tekrar tekrar üretme.”

#    Eğer modelin içinde bu attention çok kez çağrılıyorsa (özellikle backbone’da her blokta), bu tekrarlar gereksiz maliyet olur.
# Cache sayesinde:
# İlk sefer grid üretilir ve saklanır.
# Aynı H,W/device/dtype/norm tekrar gelince:
# grid yeniden hesaplanmaz,
# direkt hazır grid kullanılır.

# Daha önce ürettiğim koordinat haritalarını (x grid ve y grid) cache’den çek, tekrar hesaplama.”
        if key in self._coord_cache: # “Bu H,W/device/dtype/norm kombinasyonu için koordinat gridini daha önce üretmiş miydim?”
            xg, yg = self._coord_cache[key] # İkisini birlikte saklamak mantıklı; aynı anda üretiliyorlar, aynı koşula bağlılar.

# key var mı?
# varsa: direkt al (xg, yg = cache[key])
# yoksa: xg, yg üret

# cache’e kaydet (cache[key]=(xg,yg))
# xg: x koordinat grid’i
# yg: y koordinat grid’i

# Şekilleri genelde:
# xg.shape = (1, 1, H, W)
# yg.shape = (1, 1, H, W)
        else:
            if self.coord_norm == "minus1to1":
                xs = torch.linspace(-1.0, 1.0, W, device=device, dtype=dtype)
                ys = torch.linspace(-1.0, 1.0, H, device=device, dtype=dtype)
            else:
                xs = torch.linspace(0.0, 1.0, W, device=device, dtype=dtype)
                ys = torch.linspace(0.0, 1.0, H, device=device, dtype=dtype)

            yy, xx = torch.meshgrid(ys, xs, indexing="ij") # meshgrid: 1D xs ve ys’yi 2D (H,W) koordinat haritasına çevirir.

# ilk eksen = i = satır = y (H)
# ikinci eksen = j = sütun = x (W)


# # # # # meshgrid sadece (H,W) üretir.
# # # # # (1,1,H,W) şekli tamamen unsqueeze ile kazandırılıyor.
            xg = xx.unsqueeze(0).unsqueeze(0)  # (1,1,H,W)
            yg = yy.unsqueeze(0).unsqueeze(0)  # (1,1,H,W)
            self._coord_cache[key] = (xg, yg)

# Bizim elimizdeki cache ifadesi şuydu :: ::  (1, 1, H, W). Ama forwardda SA'ya eklemek için boyutu böyle yapmamız lazım :: :: (B, 1, H, W).Bunu aşağıdaki kod bloğu yapıyor.
        return xg.expand(B, -1, -1, -1), yg.expand(B, -1, -1, -1)
## expand ne işe yarıyor :: expand gerçek bir kopyalama yapmaz çoğu zaman.Aynı veriyi “B tane varmış gibi” gösterir.

# # # # xg’yi batch boyutunda 1 → B büyütüyor
# # # # yg’yi batch boyutunda 1 → B büyütüyor
# # # # -1 demek: “o boyutu aynen bırak” demek
# # # #  Sonuç: (1,1,H,W) → (B,1,H,W)

    def forward(self, x: torch.Tensor):
        B, C, H, W = x.shape

        # Channel squeeze maps
        avg_map = torch.mean(x, dim=1, keepdim=True)       # (B,1,H,W)
        max_map, _ = torch.max(x, dim=1, keepdim=True)     # (B,1,H,W)

        # CoordConv channels
        x_coord, y_coord = self._coords(B, H, W, x.device, x.dtype)  # (B,1,H,W) each

        # SA input
        s = torch.cat([avg_map, max_map, x_coord, y_coord], dim=1)    # (B,4,H,W)

        # Router weights
        logits = self.router(s).flatten(1)              # (B,K) ## Router, her branch’e ne kadar güveneceğini söylüyor
        rw = torch.softmax(logits, dim=1)               # (B,K)

        # Branch stack
        z = torch.stack([br(s) for br in self.branches], dim=1)  # (B,K,1,H,W) ## Branch’ler, farklı kernel ölçeklerinde maske adayları üretiyor; router da bunları ağırlıklı topluyor.

# # # # # # # self.branches içinde K tane branch var.
# # # # # # # Her br(s) şunu üretir: (B, 1, H, W)
# # # # # # #  Sonra stack(..., dim=1) diyoruz: K tane (B,1,H,W) haritasını bir araya koyuyoruz.
############# Sonuç :: z = (B, K, 1, H, W)

        wlogit = (rw[:, :, None, None, None] * z).sum(dim=1)   # (B,1,H,W) ## K tane maske adayını tek maske adayına indirmek istiyoruz:

# # # rw normalde :: (B,K)
# # # Ama z:  (B,K,1,H,W) 
# # # Çarpabilmek için rw’yi şu şekle sokuyoruz :: (B,K,1,1,1)
# # # Bunu da None ile yapıyoruz.
# # # None şunu yapar: Boyut ekler (unsqueeze gibi)
# # # Yani :: (B,K) → (B,K,1,1,1)
# # # # # # .sum(dim=1) ne? == K’yı toplayıp tek harita yapıyoruz :: (B,K,1,H,W) → (B,1,H,W)

        # Temperature + gate mask 
        T = self.get_T()
        sa = self.gate_fn(wlogit / T)                   # (B,1,H,W)
        y = x * sa

        if self.return_router_weights:
            return y, sa, rw
        return y, sa
    
# #  y: mask uygulanmış çıktı

# #  sa: maskenin kendisi (istersen dışarıda görürsün)

## 2) SA bloğu ne yapıyor?

Girdi:
- `x` ∈ ℝ^(B, C, H, W)

Çıktı:
- `sa` ∈ ℝ^(B, 1, H, W)  (spatial maske)
- `y`  ∈ ℝ^(B, C, H, W)  (yeniden ölçeklenmiş çıktı)

Temel fikir:
- “Hangi **konumlar** önemli?” sorusuna cevap verir.
- Maskeyi **multi-scale** (birden fazla kernel) ve **örnek bazlı** (router ağırlıkları) üretir.
- Konum bilgisini iyileştirmek için `CoordConv` (x/y koordinat kanalları) ekler.


## 3) Yardımcı fonksiyonlar

### 3.1) `_make_odd`
Kernel boyutunun tek (odd) olmasını garanti eder.

- Tek kernel → simetrik padding daha kolay (merkezli filtre)
- Çift kernel → “merkez” belirsizleşir; genelde istenmez

Bu fonksiyon, çift gelen kernel’i 1 artırarak tek yapar.


### 3.2) `_softplus_inverse` ve temperature mantığı
Learnable temperature gerekiyorsa T’nin pozitif kalması gerekir.

- `t_raw` serbest parametredir.
- `T = softplus(t_raw) + eps` ile T > 0 garanti edilir.
- Başlangıçta verilen `temperature` değerine eşit bir T ile başlatmak için inverse-softplus kullanılır.


### 3.3) `_get_gate`
`sigmoid` veya `hardsigmoid` seçimi yapar.
Amaç: logit uzayından (ℝ) maske uzayına ([0,1]) geçiş.


## 4) `_DWPointwiseBranch` (branch yapısı)

Her branch tek kanallı bir spatial logit üretir.

- Girdi: `s` (B,4,H,W)
- Çıkış: (B,1,H,W)

İçerik:
1) Depthwise Conv: her kanal ayrı filtrelenir (maliyet düşer)
2) Pointwise Conv: kanalları 1 kanala indirger

Bu yapı, kernel havuzunu genişletirken maliyeti kontrol eder.


## 5) `__init__`: Branch havuzu (multi-scale)

- `kernels=(3,5,7,...)` -> her biri için bir branch
- `use_dilated=True` -> ek bir dilated branch

`_make_odd`:
- kernel çiftse 1 artırılır

Dilation branch:
- `pad = dilation*(k-1)//2` ile çıktı boyutu korunur.


## 6) `__init__`: Router

Router’ın görevi:
- her örnek için branch ağırlıklarını (B,K) üretmek

Akış:
- `AdaptiveAvgPool2d(1)` -> (B,4,1,1) global özet
- `Conv(4->hidden)` + ReLU
- `Conv(hidden->K)` -> (B,K,1,1) -> flatten -> (B,K)
- softmax -> `rw` (B,K), satır toplamı 1


## 7) CoordConv grid (`_coords`) ve cache

`_coords`:
- x ve y koordinat gridlerini üretir.
- `coord_norm`:
  - `minus1to1` -> [-1,1]
  - `0to1` -> [0,1]

Cache:
- aynı (H,W,device,dtype,norm) için grid tekrar üretilmez.
- Bu performans içindir; doğruluğu değiştirmez.

Önemli: grid `x.device` ve `x.dtype` ile üretilir, dtype/device mismatch engellenir.


## 8) `forward(x)` — adım adım

### 8.1) Kanal özet haritaları
- `avg_map = mean(x, dim=1)` -> (B,1,H,W)
- `max_map = max(x, dim=1)`  -> (B,1,H,W)

Bu iki harita, her konumda kanal aktivasyonlarının özetidir.


### 8.2) CoordConv ekleme ve SA girişi
- `x_coord, y_coord` -> (B,1,H,W)
- `s = cat([avg_map, max_map, x_coord, y_coord])` -> (B,4,H,W)

Burada amaç:
- maske üretiminin konum bilgisini doğrudan kullanabilmesi.


### 8.3) Router ağırlıkları
- `logits = router(s).flatten(1)` -> (B,K)
- `rw = softmax(logits, dim=1)`   -> (B,K)

`logits` ağırlık değildir; normalize edilince `rw` olur.


### 8.4) Branch çıktıları ve ağırlıklı birleşim

- Her branch: `br(s)` -> (B,1,H,W)
- stack: `z` -> (B,K,1,H,W)

Ağırlıklı toplama:
- `rw` (B,K) -> broadcast ile (B,K,1,1,1)
- `wlogit = sum_k rw_k * z_k` -> (B,1,H,W)

Bu, örnek bazlı multi-scale birleşimidir.


### 8.5) Temperature + gate + uygulama
- `sa = gate(wlogit / T)` -> (B,1,H,W)
- `y = x * sa` -> (B,C,H,W)

CA’dan fark:
- CA kanal bazlı (B,C,1,1)
- SA konum bazlı (B,1,H,W)


## 9) Şekil özeti

- `s`: (B,4,H,W)
- `rw`: (B,K)
- `z`: (B,K,1,H,W)
- `wlogit`: (B,1,H,W)
- `sa`: (B,1,H,W)
- `y`: (B,C,H,W)
